# postmortem.ipynb

In [3]:
import pandas as pd

In [4]:
# get the data we need from the beetles table

df = pd.read_csv('../db_tables/beetles.csv')

# filter rows so that we have only those beetles used in the bioassay
df = df[~pd.isna(df.treatment)]

# filer columns so that we have only those we need
df = df[['sn','treatment','date_dosed','date_dead','pm_mites','pm_nema','pm_heartbeat']]

df

,sn,treatment,date_dosed,date_dead,pm_mites,pm_nema,pm_heartbeat
1,2,CONTROL,2023-08-24,2023-08-29,0,0,0
3,4,PNG,2023-08-24,2023-09-21,0,1,0
6,7,PNG,2023-08-24,2023-10-02,1,1,0
10,11,CONTROL,2023-08-24,2023-10-17,0,1,1
17,18,CONTROL,2023-08-15,2023-08-28,0,0,0
...,...,...,...,...,...,...,...
444,446,DUG42,2023-08-31,2023-10-19,0,0,1
447,449,PNG,2023-08-31,2023-10-19,0,0,1
449,451,X2B,2023-08-31,2023-10-19,0,0,1
452,454,V23B,2023-08-31,2023-10-19,0,0,1


In [5]:
# here is a list of serial nummbers for beetles alive at the end of the bioassay
# date_dead should be set to "0000-00-00" for these beetles

alive_at_end_of_bioassay =[364, 369, 375, 376, 383, 386, 392, 400, 416, 423, 437, 445, 446, 449, 451, 454, 11, 208, 209, 107,
                           143, 177, 197, 104, 118, 47, 43, 88]

for sn in alive_at_end_of_bioassay:
    df.loc[df['sn']==sn, 'date_dead'] = '0000-00-00'

In [35]:
# create a table of percent of beetles with mites, nematodes and heartbeat

df_count = df[df.date_dead != '0000-00-00'].groupby(['treatment'])[['pm_mites','pm_nema','pm_heartbeat']].count().reset_index()
df_sum = df[df.date_dead != '0000-00-00'].groupby(['treatment'])[['pm_mites','pm_nema','pm_heartbeat']].sum().reset_index()
df1 = df_count.merge(df_sum, on='treatment')
df1['mites_pct'] = (100 * (df1.pm_mites_y / df1.pm_mites_x)).astype(int)
df1['nema_pct'] = (100 * (df1.pm_nema_y / df1.pm_nema_x)).astype(int)
df1['heartbeat_pct'] = (100 * (df1.pm_heartbeat_y / df1.pm_heartbeat_x)).astype(int)
df1 = df1[['treatment','mites_pct','nema_pct','heartbeat_pct']]
df1

,treatment,mites_pct,nema_pct,heartbeat_pct
0,CONTROL,25,50,50
1,DUG42,33,66,46
2,PNG,22,63,40
3,V23B,26,61,50
4,X2B,9,54,40


In [39]:
print(df1.to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
treatment &  mites\_pct &  nema\_pct &  heartbeat\_pct \\
\midrule
  CONTROL &         25 &        50 &             50 \\
    DUG42 &         33 &        66 &             46 \\
      PNG &         22 &        63 &             40 \\
     V23B &         26 &        61 &             50 \\
      X2B &          9 &        54 &             40 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2572352/1468421615.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df1.to_latex(index=False))
